# 8. Trial of Training Discrete Model （One model for each DOLocation)

- Trains a selection of classification models with default hyperparameter values to gauge the time required to train-score one model, and also get a preview of the accuracy scores. However this time try train-scoring one model for each DOLocation and make final score = weighted sum of each DOLocation's model's score
- notebook follows Initialise, Train, Predict, Get Accuracy Score steps

## Create Function for special evaluation

In [1]:
import pandas as pd

In [2]:
def accuracy_count(pred, obs):
    """ Special function for evaluating observation - but here return count of correctly predicted samples instead of accuracy score """
    n = len(pred)
    n_tp = 0

    for i in range(n):
        if pred[i] in obs[i]:
            n_tp += 1

    return n_tp

# Import Data

In [4]:
Train_data_X = pd.read_csv('../data/curated/ModelBuilding/Discrete/XTrain_16-1_16-5.csv')
Train_data_y = pd.read_csv('../data/curated/ModelBuilding/Discrete/yTrain_16-1_16-5.csv')
XTrain = Train_data_X.drop(['datetime'], axis=1)
yTrain = Train_data_y['Max_PULocationID']

In [5]:
Val_data_X = pd.read_csv('../data/curated/ModelBuilding/Discrete/XVal_16-5_16-6.csv')
Val_data_y = pd.read_csv('../data/curated/ModelBuilding/Discrete/yVal_16-5_16-6.csv')
XVal = Val_data_X.drop(['datetime'], axis=1)

In [6]:
Test_data_X = pd.read_csv('../data/curated/ModelBuilding/Discrete/XTest_16-6_16-6.csv')
Test_data_y = pd.read_csv('../data/curated/ModelBuilding/Discrete/yTest_16-6_16-6.csv')
XTest = Test_data_X.drop(['datetime'], axis=1)

# Models
## 0R

In [7]:
from sklearn.dummy import DummyClassifier

In [8]:
# Stores number of correct predictions
train_correct = 0
val_correct = 0
test_correct = 0


for i in set(XTrain['DOLocationID']):
    print(i)

    # first subset the data to only contain rows for which
    # DOLocation = the location it has iterated to
    XTrain_subset = XTrain[XTrain['DOLocationID']==i]
    yTrain_subset = yTrain[XTrain['DOLocationID']==i]

    yTrain_subset_full = Train_data_y[XTrain['DOLocationID']==i]

    XVal_subset = XVal[XVal['DOLocationID']==i]
    yVal_subset = Val_data_y[XVal['DOLocationID']==i]

    XTest_subset = XTest[XTest['DOLocationID']==i]
    yTest_subset = Test_data_y[XTest['DOLocationID']==i]

    # create the input for the accuracy count function
    # (convert what was stored as a string in csv back into a list)
    train_obs_subset = [[int(x) for x in Max_IDs.strip('[]').split(',')]
                        for Max_IDs in Train_data_y['Max_PULocationIDs']]
    val_obs_subset = [[int(x) for x in Max_IDs.strip('[]').split(',')]
                      for Max_IDs in yVal_subset['Max_PULocationIDs']]
    test_obs_subset = [[int(x) for x in Max_IDs.strip('[]').split(',')]
                       for Max_IDs in yTest_subset['Max_PULocationIDs']]

    # Train model and add the correctly predicted counts onto the total
    zero_r = DummyClassifier()
    zero_r.fit(XTrain_subset, yTrain_subset)
    train_pred_0R = zero_r.predict(XTrain_subset)
    train_correct += accuracy_count(train_pred_0R, train_obs_subset)

    val_pred_0R = zero_r.predict(XVal_subset)
    val_correct += accuracy_count(val_pred_0R, val_obs_subset)

    test_pred_0R = zero_r.predict(XTest_subset)
    test_correct += accuracy_count(test_pred_0R, test_obs_subset)

1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
24.0
25.0
26.0
28.0
29.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
53.0
54.0
55.0
56.0
57.0
59.0
60.0
61.0
62.0
63.0
64.0
65.0
66.0
67.0
68.0
69.0
70.0
71.0
72.0
73.0
74.0
75.0
76.0
77.0
78.0
79.0
80.0
82.0
83.0
85.0
86.0
87.0
88.0
89.0
90.0
91.0
92.0
93.0
94.0
95.0
96.0
97.0
98.0
99.0
100.0
101.0
102.0
106.0
107.0
108.0
111.0
112.0
113.0
114.0
115.0
116.0
117.0
118.0
119.0
120.0
121.0
122.0
123.0
124.0
125.0
126.0
127.0
128.0
129.0
130.0
131.0
132.0
133.0
134.0
135.0
136.0
137.0
138.0
139.0
140.0
141.0
142.0
143.0
144.0
145.0
146.0
147.0
148.0
149.0
150.0
151.0
152.0
153.0
154.0
155.0
157.0
158.0
159.0
160.0
161.0
162.0
163.0
164.0
165.0
166.0
167.0
168.0
169.0
170.0
171.0
173.0
174.0
175.0
177.0
178.0
179.0
180.0
181.0
182.0
183.0
185.0
186.0
188.0
189.0
190.0
191.0
192.0
193.0
194.0
195.0
196.0
197.0
198.0
199.0
200.0
201

In [9]:

# get the final accuracy for Train Val Test
print(train_correct/len(XTrain))
print(val_correct/len(XVal))
print(test_correct/len(XTest))

0.03650233044867161
0.5526234726357943
0.5552276408133885


## XGB
*All commenting from hereonin as per 0R*

In [8]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [9]:
train_correct = 0
val_correct = 0
test_correct = 0

for i in set(XTrain['DOLocationID']):
    print(i)

    XTrain_subset = XTrain[XTrain['DOLocationID'] == i]
    yTrain_subset = yTrain[XTrain['DOLocationID'] == i]

    yTrain_subset_full = Train_data_y[XTrain['DOLocationID'] == i]

    XVal_subset = XVal[XVal['DOLocationID'] == i]
    yVal_subset = Val_data_y[XVal['DOLocationID'] == i]

    XTest_subset = XTest[XTest['DOLocationID'] == i]
    yTest_subset = Test_data_y[XTest['DOLocationID'] == i]

    train_obs_subset = [[int(x) for x in Max_IDs.strip('[]').split(',')]
                        for Max_IDs in Train_data_y['Max_PULocationIDs']]
    val_obs_subset = [[int(x) for x in Max_IDs.strip('[]').split(',')]
                      for Max_IDs in yVal_subset['Max_PULocationIDs']]
    test_obs_subset = [[int(x) for x in Max_IDs.strip('[]').split(',')]
                       for Max_IDs in yTest_subset['Max_PULocationIDs']]

    XGB = xgb.XGBClassifier(colsample_bytree=1, max_depth = 8)
    if len(set(yTrain_subset)) == 1:
        continue

    # Extra transformation of labels for XGB requirements
    le = LabelEncoder()
    yTrain_subset_XGB = le.fit_transform(yTrain_subset)

    XGB.fit(XTrain_subset, yTrain_subset_XGB)
    train_pred_XGB_untransformed = XGB.predict(XTrain_subset)
    train_pred_XGB = le.inverse_transform(train_pred_XGB_untransformed)
    train_correct += accuracy_count(train_pred_XGB_untransformed,
                                    train_obs_subset)

    if len(XVal_subset) > 0:

        val_pred_XGB_untransformed = XGB.predict(XVal_subset)
        val_pred_XGB = le.inverse_transform(val_pred_XGB_untransformed)

        val_correct += accuracy_count(val_pred_XGB_untransformed,
                                      val_obs_subset)

    if len(XTest_subset) > 0:

        test_pred_XGB_untransformed = XGB.predict(XTest_subset)
        test_pred_XGB = le.inverse_transform(test_pred_XGB_untransformed)

        test_correct += accuracy_count(test_pred_XGB, test_obs_subset)

1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
23.0
24.0
25.0
26.0
27.0
28.0
29.0
30.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
44.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
53.0
54.0
55.0
56.0
57.0
58.0
59.0
60.0
61.0
62.0
63.0
64.0
65.0
66.0
67.0
68.0
69.0
70.0
71.0
72.0
73.0
74.0
75.0
76.0
77.0
78.0
79.0
80.0
81.0
82.0
83.0
84.0
85.0
86.0
87.0
88.0
89.0
90.0
91.0
92.0
93.0
94.0
95.0
96.0
97.0
98.0
99.0
100.0
101.0
102.0
106.0
107.0
108.0
109.0
111.0
112.0
113.0
114.0
115.0
116.0
117.0
118.0
119.0
120.0
121.0
122.0
123.0
124.0
125.0
126.0
127.0
128.0
129.0
130.0
131.0
132.0
133.0
134.0
135.0
136.0
137.0
138.0
139.0
140.0
141.0
142.0
143.0
144.0
145.0
146.0
147.0
148.0
149.0
150.0
151.0
152.0
153.0
154.0
155.0
156.0
157.0
158.0
159.0
160.0
161.0
162.0
163.0
164.0
165.0
166.0
167.0
168.0
169.0
170.0
171.0
172.0
173.0
174.0
175.0
176.0
177.0
178.0
179.0
180.0
181.0
182.0
183.0
184.0
185.0
186.0
187.0
188.0
189.

In [10]:
print(train_correct / len(XTrain))
print(val_correct / len(XVal))
print(test_correct / len(XTest))

0.006227095739807604
0.003366031929217157
0.030453257790368272


## NB

In [10]:
from sklearn.naive_bayes import GaussianNB

train_correct = 0
val_correct = 0
test_correct = 0

for i in set(XTrain['DOLocationID']):
    print(i)

    XTrain_subset = XTrain[XTrain['DOLocationID'] == i]
    yTrain_subset = yTrain[XTrain['DOLocationID'] == i]

    yTrain_subset_full = Train_data_y[XTrain['DOLocationID'] == i]

    XVal_subset = XVal[XVal['DOLocationID'] == i]
    yVal_subset = Val_data_y[XVal['DOLocationID'] == i]

    XTest_subset = XTest[XTest['DOLocationID'] == i]
    yTest_subset = Test_data_y[XTest['DOLocationID'] == i]

    train_obs_subset = [[int(x) for x in Max_IDs.strip('[]').split(',')]
                        for Max_IDs in Train_data_y['Max_PULocationIDs']]
    val_obs_subset = [[int(x) for x in Max_IDs.strip('[]').split(',')]
                      for Max_IDs in yVal_subset['Max_PULocationIDs']]
    test_obs_subset = [[int(x) for x in Max_IDs.strip('[]').split(',')]
                       for Max_IDs in
                       yTest_subset['Max_PULocationIDs']]

    GNB = GaussianNB()
    if len(set(yTrain_subset)) == 1:
        continue

    GNB.fit(XTrain_subset, yTrain_subset)
    train_pred_GNB = GNB.predict(XTrain_subset)
    train_correct += accuracy_count(train_pred_GNB, train_obs_subset)

    if len(XVal_subset) > 0:
        val_pred_GNB = GNB.predict(XVal_subset)
        val_correct += accuracy_count(val_pred_GNB, val_obs_subset)

    if len(XTest_subset) > 0:
        test_pred_GNB = GNB.predict(XTest_subset)
        test_correct += accuracy_count(test_pred_GNB, test_obs_subset)

1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
24.0
25.0
26.0
28.0
29.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
53.0
54.0
55.0
56.0
57.0
59.0
60.0
61.0
62.0
63.0
64.0
65.0
66.0
67.0
68.0
69.0
70.0
71.0
72.0
73.0
74.0
75.0
76.0
77.0
78.0
79.0
80.0
82.0
83.0
85.0
86.0
87.0
88.0
89.0
90.0
91.0
92.0
93.0
94.0
95.0
96.0
97.0
98.0
99.0
100.0
101.0
102.0
106.0
107.0
108.0
111.0
112.0
113.0
114.0
115.0
116.0
117.0
118.0
119.0
120.0
121.0
122.0
123.0
124.0
125.0
126.0
127.0
128.0
129.0
130.0
131.0
132.0
133.0
134.0
135.0
136.0
137.0
138.0
139.0
140.0
141.0
142.0
143.0
144.0
145.0
146.0
147.0
148.0
149.0
150.0
151.0
152.0
153.0
154.0
155.0
157.0
158.0
159.0
160.0
161.0
162.0
163.0
164.0
165.0
166.0
167.0
168.0
169.0
170.0
171.0
173.0
174.0
175.0
177.0
178.0
179.0
180.0
181.0
182.0
183.0
185.0
186.0
188.0
189.0
190.0
191.0
192.0
193.0
194.0
195.0
196.0
197.0
198.0
199.0
200.0
201

In [11]:
print(train_correct / len(XTrain))
print(val_correct / len(XVal))
print(test_correct / len(XTest))

0.02447487452754198
0.1778758941712017
0.1762461759942415


---

In [12]:
from sklearn.naive_bayes import BernoulliNB

train_correct = 0
val_correct = 0
test_correct = 0

for i in set(XTrain['DOLocationID']):
    print(i)

    XTrain_subset = XTrain[XTrain['DOLocationID'] == i]
    yTrain_subset = yTrain[XTrain['DOLocationID'] == i]

    yTrain_subset_full = Train_data_y[XTrain['DOLocationID'] == i]

    XVal_subset = XVal[XVal['DOLocationID'] == i]
    yVal_subset = Val_data_y[XVal['DOLocationID'] == i]

    XTest_subset = XTest[XTest['DOLocationID'] == i]
    yTest_subset = Test_data_y[XTest['DOLocationID'] == i]

    train_obs_subset = [[int(x) for x in Max_IDs.strip('[]').split(',')]
                        for Max_IDs in Train_data_y['Max_PULocationIDs']]
    val_obs_subset = [[int(x) for x in Max_IDs.strip('[]').split(',')]
                      for Max_IDs in yVal_subset['Max_PULocationIDs']]
    test_obs_subset = [[int(x) for x in Max_IDs.strip('[]').split(',')]
                       for Max_IDs in yTest_subset['Max_PULocationIDs']]

    MNB = BernoulliNB()
    if len(set(yTrain_subset)) == 1:
        continue

    MNB.fit(XTrain_subset, yTrain_subset)
    train_pred_MNB = MNB.predict(XTrain_subset)
    train_correct += accuracy_count(train_pred_MNB, train_obs_subset)

    if len(XVal_subset) > 0:
        val_pred_MNB = MNB.predict(XVal_subset)
        val_correct += accuracy_count(val_pred_MNB, val_obs_subset)

    if len(XTest_subset) > 0:
        test_pred_MNB = MNB.predict(XTest_subset)
        test_correct += accuracy_count(test_pred_MNB, test_obs_subset)

1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
24.0
25.0
26.0
28.0
29.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
53.0
54.0
55.0
56.0
57.0
59.0
60.0
61.0
62.0
63.0
64.0
65.0
66.0
67.0
68.0
69.0
70.0
71.0
72.0
73.0
74.0
75.0
76.0
77.0
78.0
79.0
80.0
82.0
83.0
85.0
86.0
87.0
88.0
89.0
90.0
91.0
92.0
93.0
94.0
95.0
96.0
97.0
98.0
99.0
100.0
101.0
102.0
106.0
107.0
108.0
111.0
112.0
113.0
114.0
115.0
116.0
117.0
118.0
119.0
120.0
121.0
122.0
123.0
124.0
125.0
126.0
127.0
128.0
129.0
130.0
131.0
132.0
133.0
134.0
135.0
136.0
137.0
138.0
139.0
140.0
141.0
142.0
143.0
144.0
145.0
146.0
147.0
148.0
149.0
150.0
151.0
152.0
153.0
154.0
155.0
157.0
158.0
159.0
160.0
161.0
162.0
163.0
164.0
165.0
166.0
167.0
168.0
169.0
170.0
171.0
173.0
174.0
175.0
177.0
178.0
179.0
180.0
181.0
182.0
183.0
185.0
186.0
188.0
189.0
190.0
191.0
192.0
193.0
194.0
195.0
196.0
197.0
198.0
199.0
200.0
201

In [13]:
print(train_correct / len(XTrain))
print(val_correct / len(XVal))
print(test_correct / len(XTest))

0.03719767850135283
0.6439743984666462
0.6482634515026093


## logR

In [14]:
from sklearn.linear_model import LogisticRegression

In [15]:
train_correct = 0
val_correct = 0
test_correct = 0


for i in set(XTrain['DOLocationID']):
    print(i)

    XTrain_subset = XTrain[XTrain['DOLocationID']==i]
    yTrain_subset = yTrain[XTrain['DOLocationID']==i]

    yTrain_subset_full = Train_data_y[XTrain['DOLocationID']==i]

    XVal_subset = XVal[XVal['DOLocationID']==i]
    yVal_subset = Val_data_y[XVal['DOLocationID']==i]

    XTest_subset = XTest[XTest['DOLocationID']==i]
    yTest_subset = Test_data_y[XTest['DOLocationID']==i]

    train_obs_subset = [[int(x) for x in Max_IDs.strip('[]').split(',')]
                        for Max_IDs in Train_data_y['Max_PULocationIDs']]
    val_obs_subset = [[int(x) for x in Max_IDs.strip('[]').split(',')]
                      for Max_IDs in yVal_subset['Max_PULocationIDs']]
    test_obs_subset = [[int(x) for x in Max_IDs.strip('[]').split(',')]
                       for Max_IDs in yTest_subset['Max_PULocationIDs']]

    logR = LogisticRegression(max_iter=1000)
    if len(set(yTrain_subset)) == 1:
        continue

    logR.fit(XTrain_subset, yTrain_subset)
    train_pred_logR = logR.predict(XTrain_subset)
    train_correct += accuracy_count(train_pred_logR, train_obs_subset)

    if len(XVal_subset) > 0:
        val_pred_logR = logR.predict(XVal_subset)
        val_correct += accuracy_count(val_pred_logR, val_obs_subset)

    if len(XTest_subset) > 0:
        test_pred_logR = logR.predict(XTest_subset)
        test_correct += accuracy_count(test_pred_logR, test_obs_subset)

1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
24.0
25.0
26.0
28.0
29.0
31.0
32.0
33.0


/Applications/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


34.0
35.0
36.0


/Applications/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


37.0


...

/Applications/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


262.0


/Applications/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


263.0


/Applications/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [16]:
print(train_correct/len(XTrain))
print(val_correct/len(XVal))
print(test_correct/len(XTest))

0.03856083606997542
0.6534894068521752
0.6544178513586467


## RF

In [17]:
from sklearn.ensemble import RandomForestClassifier

In [18]:
train_correct = 0
val_correct = 0
test_correct = 0


for i in set(XTrain['DOLocationID']):
    print(i)

    XTrain_subset = XTrain[XTrain['DOLocationID']==i]
    yTrain_subset = yTrain[XTrain['DOLocationID']==i]

    yTrain_subset_full = Train_data_y[XTrain['DOLocationID']==i]

    XVal_subset = XVal[XVal['DOLocationID']==i]
    yVal_subset = Val_data_y[XVal['DOLocationID']==i]

    XTest_subset = XTest[XTest['DOLocationID']==i]
    yTest_subset = Test_data_y[XTest['DOLocationID']==i]

    train_obs_subset = [[int(x) for x in Max_IDs.strip('[]').split(',')]
                        for Max_IDs in Train_data_y['Max_PULocationIDs']]
    val_obs_subset = [[int(x) for x in Max_IDs.strip('[]').split(',')]
                      for Max_IDs in yVal_subset['Max_PULocationIDs']]
    test_obs_subset = [[int(x) for x in Max_IDs.strip('[]').split(',')]
                       for Max_IDs in yTest_subset['Max_PULocationIDs']]

    RF = RandomForestClassifier(n_estimators = 100, max_depth=16)
    RF.fit(XTrain_subset, yTrain_subset)
    train_pred_RF = RF.predict(XTrain_subset)
    train_correct += accuracy_count(train_pred_RF, train_obs_subset)

    if len(XVal_subset) > 0:
        val_pred_RF = RF.predict(XVal_subset)
        val_correct += accuracy_count(val_pred_RF, val_obs_subset)

    if len(XTest_subset) > 0:
        test_pred_RF = RF.predict(XTest_subset)
        test_correct += accuracy_count(test_pred_RF, test_obs_subset)

1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
24.0
25.0
26.0
28.0
29.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
53.0
54.0
55.0
56.0
57.0
59.0
60.0
61.0
62.0
63.0
64.0
65.0
66.0
67.0
68.0
69.0
70.0
71.0
72.0
73.0
74.0
75.0
76.0
77.0
78.0
79.0
80.0
82.0
83.0
85.0
86.0
87.0
88.0
89.0
90.0
91.0
92.0
93.0
94.0
95.0
96.0
97.0
98.0
99.0
100.0
101.0
102.0
106.0
107.0
108.0
111.0
112.0
113.0
114.0
115.0
116.0
117.0
118.0
119.0
120.0
121.0
122.0
123.0
124.0
125.0
126.0
127.0
128.0
129.0
130.0
131.0
132.0
133.0
134.0
135.0
136.0
137.0
138.0
139.0
140.0
141.0
142.0
143.0
144.0
145.0
146.0
147.0
148.0
149.0
150.0
151.0
152.0
153.0
154.0
155.0
157.0
158.0
159.0
160.0
161.0
162.0
163.0
164.0
165.0
166.0
167.0
168.0
169.0
170.0
171.0
173.0
174.0
175.0
177.0
178.0
179.0
180.0
181.0
182.0
183.0
185.0
186.0
188.0
189.0
190.0
191.0
192.0
193.0
194.0
195.0
196.0
197.0
198.0
199.0
200.0
201

In [19]:
print(train_correct/len(XTrain))
print(val_correct/len(XVal))
print(test_correct/len(XTest))

0.036812138986994924
0.6127596946982921
0.5894547417671405


# SVM

In [20]:
from sklearn.svm import SVC

train_correct = 0
val_correct = 0
test_correct = 0

for i in set(XTrain['DOLocationID']):
    print(i)

    XTrain_subset = XTrain[XTrain['DOLocationID'] == i]
    yTrain_subset = yTrain[XTrain['DOLocationID'] == i]

    yTrain_subset_full = Train_data_y[XTrain['DOLocationID'] == i]

    XVal_subset = XVal[XVal['DOLocationID'] == i]
    yVal_subset = Val_data_y[XVal['DOLocationID'] == i]

    XTest_subset = XTest[XTest['DOLocationID'] == i]
    yTest_subset = Test_data_y[XTest['DOLocationID'] == i]

    train_obs_subset = [[int(x) for x in Max_IDs.strip('[]').split(',')]
                        for Max_IDs in Train_data_y['Max_PULocationIDs']]
    val_obs_subset = [[int(x) for x in Max_IDs.strip('[]').split(',')]
                      for Max_IDs in yVal_subset['Max_PULocationIDs']]
    test_obs_subset = [[int(x) for x in Max_IDs.strip('[]').split(',')]
                       for Max_IDs in yTest_subset['Max_PULocationIDs']]

    svm = SVC()
    if len(set(yTrain_subset)) == 1:
        continue

    svm.fit(XTrain_subset, yTrain_subset)
    train_pred_svm = svm.predict(XTrain_subset)
    train_correct += accuracy_count(train_pred_svm, train_obs_subset)

    if len(XVal_subset) > 0:
        val_pred_svm = svm.predict(XVal_subset)
        val_correct += accuracy_count(val_pred_svm, val_obs_subset)

    if len(XTest_subset) > 0:
        test_pred_svm = svm.predict(XTest_subset)
        test_correct += accuracy_count(test_pred_svm, test_obs_subset)

1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
24.0
25.0
26.0
28.0
29.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
53.0
54.0
55.0
56.0
57.0
59.0
60.0
61.0
62.0
63.0
64.0
65.0
66.0
67.0
68.0
69.0
70.0
71.0
72.0
73.0
74.0
75.0
76.0
77.0
78.0
79.0
80.0
82.0
83.0
85.0
86.0
87.0
88.0
89.0
90.0
91.0
92.0
93.0
94.0
95.0
96.0
97.0
98.0
99.0
100.0
101.0
102.0
106.0
107.0
108.0
111.0
112.0
113.0
114.0
115.0
116.0
117.0
118.0
119.0
120.0
121.0
122.0
123.0
124.0
125.0
126.0
127.0
128.0
129.0
130.0
131.0
132.0
133.0
134.0
135.0
136.0
137.0
138.0
139.0
140.0
141.0
142.0
143.0
144.0
145.0
146.0
147.0
148.0
149.0
150.0
151.0
152.0
153.0
154.0
155.0
157.0
158.0
159.0
160.0
161.0
162.0
163.0
164.0
165.0
166.0
167.0
168.0
169.0
170.0
171.0
173.0
174.0
175.0
177.0
178.0
179.0
180.0
181.0
182.0
183.0
185.0
186.0
188.0
189.0
190.0
191.0
192.0
193.0
194.0
195.0
196.0
197.0
198.0
199.0
200.0
201

In [21]:
print(train_correct/len(XTrain))
print(val_correct/len(XVal))
print(test_correct/len(XTest))

0.036357753130787394
0.5498511140774207
0.5516645672125248
